In [1]:
import os
import torch
import glob
from torchvision import models, transforms
from PIL import Image, ImageDraw
import torch.nn as nn
from annoy import AnnoyIndex

In [2]:
test_images_folder = '../cropped_segments'
test_images = os.listdir(test_images_folder)
len(test_images)

190

In [3]:
database_images= glob.glob('../database/*.jpg')
len(database_images)

689

In [4]:
weights = models.MobileNet_V3_Large_Weights.IMAGENET1K_V2
model = models.mobilenet_v3_large(weights=weights)
model.classifier = nn.Identity()
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [5]:
transform = transforms.Compose([
    transforms.Resize((640,640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
annoy_index = AnnoyIndex(960, 'angular')
annoy_index.load('productDbIndex_MobileNet_V3_Large_Weights.ann')

True

In [7]:
image_grid = Image.new('RGB', (400,200))

In [8]:
lis = []
for i in range(len(test_images)):
    image_path = os.path.join(test_images_folder, test_images[i])
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0)
    output_tensor = model(image_tensor)

    nns, distance = annoy_index.get_nns_by_vector(output_tensor[0], 1, include_distances=True)
    lis.append([image_path, nns[0],distance[0]])

    image = image.resize((200,200))
    image_draw = ImageDraw.Draw(image)
    image_draw.rectangle([(0,0),(199,199)], outline='red', width=8)
    image_grid.paste(image, ((0,0)))

    search_image = Image.open(database_images[nns[0]]).resize((200,200))
    image_grid.paste(search_image, ((200,0)))
    image_grid.save(f'../dumpImage/image_{i}_{distance}.png')